In [ ]:
# check everything is set up and ready
import sklearn
import numpy
import xgboost

In [ ]:
import pickle

with open('matches.pickle', 'rb') as f:
    matches = pickle.loads(f.read())

In [ ]:
# do some exploratory analysis here


In [ ]:
y = [] # the correct labels
baseline_predictions = []

# write code here...


# .. so that the asserts pass
assert len(y) == len(baseline_predictions) == len(matches)
assert sum(y) == 15739
assert sum(baseline_predictions) == 37010

In [ ]:
import numpy as np

y = np.array(y)
baseline_predictions = np.array(baseline_predictions)

sum(baseline_predictions == y) / len(y)

In [ ]:
with open('embeddings.pickle', 'rb') as f:
  member_embeddings, request_embeddings = pickle.loads(f.read())

In [ ]:
from sklearn.model_selection import train_test_split

X = []
for match in matches:
    X.append(member_embeddings[match['member_id']] * \
        request_embeddings[match['request_id']])

X = np.array(X)

X_train, X_test, y_train, y_test, _, baseline_predictions_test = \
    train_test_split(X, y, baseline_predictions, test_size=0.05, random_state=42)

In [ ]:
from xgboost import XGBClassifier

model_v1 = XGBClassifier(
    n_estimators=5, # <- if on CPU, reduce this to.. 5?
    max_depth=7,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='auc',
    #gpu_id=0, # if you don't have a GPU set up, remove this and run on CPU
    #tree_method='gpu_hist', # if on CPU, remove this
)
model_v1.fit(X_train, y_train)
model_v1_predictions = model_v1.predict(X_test)

In [ ]:
print('baseline:', sum(baseline_predictions_test == y_test)/len(y_test))
print('new model:', sum(model_v1_predictions == y_test)/len(y_test))

In [ ]:
EXTEND_COUNT = 120000
# list of dictionaries {'member_id': int, 'request_id': int}
true_negatives = []

# generate 120000 random new matches ...


# ... such that these tests pass
assert len(true_negatives) == EXTEND_COUNT
true_negatives_set = {(tn['member_id'], tn['request_id']) for tn in true_negatives}
assert len(true_negatives_set) == EXTEND_COUNT

old_matches_set = {(d['member_id'], d['request_id']) for d in matches}
member_ids_set = {d['member_id'] for d in matches}
request_ids_set = {d['request_id'] for d in matches}

assert len(true_negatives_set.difference(old_matches_set)) == EXTEND_COUNT
for tn in true_negatives:
    assert tn['member_id'] in member_ids_set
    assert tn['request_id'] in request_ids_set

In [ ]:
X_tn = []
y_tn = []
baseline_predictions_tn = []

# generate a dataset solely for these new true negatives...


# .. so that the asserts pass
assert len(X_tn) == len(y_tn) == len(baseline_predictions_tn)

In [ ]:
# split this new dataset into train and test...


# ... then extend the previous dataset...


# ... so that the asserts pass
assert X_train_extended.shape == (151405, 768)
assert X_test_extended.shape == (7969, 768)
assert len(y_train_extended) == 151405
assert len(y_test_extended) == 7969
assert len(baseline_predictions_test_extended) == 7969
assert sum(y_train_extended) == 14913
assert sum(y_test_extended) == 826
assert sum(baseline_predictions_test_extended) == 1829

In [ ]:
from xgboost import XGBClassifier

# use the same model setup as above regarding GPU/CPU
model_v2 = XGBClassifier(
    n_estimators=5, # <- if on CPU, reduce this to.. 5?
    max_depth=7,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='auc',
    #gpu_id=0, # if you don't have a GPU set up, remove this and run on CPU
    #tree_method='gpu_hist', # if on CPU, remove this
)
model_v2.fit(X_train_extended, y_train_extended)

In [ ]:
model_v1_predictions = model_v1.predict(X_test_extended)
model_v2_predictions = model_v2.predict(X_test_extended)

print('baseline:', sum(baseline_predictions_test_extended == y_test_extended)/len(y_test_extended))
print('model v1:', sum(model_v1_predictions == y_test_extended)/len(y_test_extended))
print('model v2:', sum(model_v2_predictions == y_test_extended)/len(y_test_extended))

In [ ]:
from sklearn import metrics

model_v1_predictions = model_v1.predict_proba(X_test_extended)[:,1]
model_v2_predictions = model_v2.predict_proba(X_test_extended)[:,1]

print('baseline:', metrics.roc_auc_score(y_test_extended, baseline_predictions_test_extended))
print('model v1:', metrics.roc_auc_score(y_test_extended, model_v1_predictions))
print('model v2:', metrics.roc_auc_score(y_test_extended, model_v2_predictions))